# Running experiments from .yaml and saving results in .csv

In [1]:
import yaml
import csv
import torch
import numpy as np

In [2]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(128, 128)
        self.conv2 = GCNConv(128, 16)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [3]:
from measures import Result


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.set_printoptions(precision=2)
np.set_printoptions(precision=2)

with open('experiments.yaml', 'r') as yaml_file:
    experiments = yaml.safe_load(yaml_file)

with open('results.csv', 'w', newline='') as csv_file:
    fieldnames = ['Dataset', 'AP', 'AF']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for experiment in experiments['experiments']:
        print(experiment['name'], experiment['directory'])
        for i in range(3):
            csbm = torch.load(experiment['directory'] + experiment['name'] + f'{i}.pt')
            result = Result(GCN().to(device), csbm)
            result.learn()
            ap = result.get_average_accuracy()
            af = 0 if experiment['name'] == 'zero_' else result.get_average_forgetting_measure()
            writer.writerow({'Dataset': f'{experiment["name"]}', 'AP': f'{ap}', 'AF': f'{af}'})
            print(f'AP: {ap:.2f}'.ljust(10) + f' AF: {af:.2f}')
        print()

cuda
base_ ./csbm_base/
AP: 0.32   AF: 0.24
AP: 0.34   AF: 0.24
AP: 0.36   AF: 0.29

zero_ ./csbm_zero/
AP: 0.38   AF: 0.00
AP: 0.38   AF: 0.00
AP: 0.30   AF: 0.00

feat_ ./csbm_feat/
AP: 0.16   AF: 0.36
AP: 0.28   AF: 0.33
AP: 0.16   AF: 0.36

struct_ ./csbm_struct/
AP: 0.22   AF: 0.36
AP: 0.30   AF: 0.27
AP: 0.38   AF: 0.11

hom_ ./csbm_hom/
AP: 0.34   AF: 0.29
AP: 0.40   AF: 0.24
AP: 0.38   AF: 0.22

class_ ./csbm_class/
AP: 0.36   AF: 0.33
AP: 0.30   AF: 0.44
AP: 0.42   AF: 0.27



In [5]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.3f}')
performance = pd.read_csv('results.csv')
print(performance.groupby('Dataset').mean())

           AP    AF
Dataset            
base_   0.340 0.259
class_  0.360 0.348
feat_   0.200 0.348
hom_    0.373 0.252
struct_ 0.300 0.244
zero_   0.353 0.000
